In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Virtual Try-On on Vertex AI


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fgetting-started%2Fvirtual_try_on.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/getting-started/virtual_try_on.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| Author |
| --- |
| [Jorj Ismailyan](https://github.com/jismailyan-google) |

## Overview

Virtual Try-On uses Google's cutting edge image generation models to generate high quality images of clothing products.


### Objectives

In this notebook, you will be exploring the features of Virtual Try-On using the Vertex AI Python SDK. You will

- Generate images by providing images of a person and a product to try on
- Supported products
 * Tops (t-shirts, button-up shirts, hoodies, sweaters, tank tops)
 * Bottoms (pants, jeans, skirts)
 * Shoes (sneakers, boots, sandals, flats, heels, dress shoes)

### Costs

- This notebook uses billable components of Google Cloud:
  - Vertex AI

- Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK for Python (Jupyter only)

In [ ]:
%pip install --upgrade --user google-cloud-aiplatform

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and enable the Vertex AI API.

Learn more about setting up a project and a development environment.

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic import PredictResponse

PROJECT_ID = ""  # @param {type:"string"}
LOCATION = "us-central1"  # @param ["us-central1"]

aiplatform.init(project=PROJECT_ID, location=LOCATION)

api_regional_endpoint = f"{LOCATION}-aiplatform.googleapis.com"
client_options = {"api_endpoint": api_regional_endpoint}
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

model_endpoint = f"projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/virtual-try-on-exp-05-31"
print(f"Prediction client initiated on project {PROJECT_ID} in {LOCATION}.")

In [ ]:
# @title Import libraries and define utilities
# @markdown Run this cell before proceeding to import libraries and define utility functions.
import base64
import io
import re
import timeit

from PIL import Image
from google.cloud import storage
from google.colab import files
import matplotlib.pyplot as plt


# Parses the generated image bytes from the response and converts it
# to a PIL Image object.
def prediction_to_pil_image(
    prediction: PredictResponse, size=(640, 640)
) -> Image.Image:
    encoded_bytes_string = prediction["bytesBase64Encoded"]
    decoded_image_bytes = base64.b64decode(encoded_bytes_string)
    image_pil = Image.open(io.BytesIO(decoded_image_bytes))
    image_pil.thumbnail(size)
    return image_pil


# Displays images and predictions in a horizontal row.
def display_row(items: list, figsize: tuple[int, int] = (15, 15)):
    count = len(items)

    if count == 0:
        print("No items to display.")
        return

    fig, ax = plt.subplots(1, count, figsize=figsize)
    if count == 1:
        axes = [ax]
    else:
        axes = ax

    for i in range(count):
        item = items[i]
        current_ax = axes[i]

        if isinstance(item, Image.Image):
            current_ax.imshow(item, None)
            current_ax.axis("off")
        elif "bytesBase64Encoded" in item:
            pil_image = prediction_to_pil_image(item)
            current_ax.imshow(pil_image, None)
            current_ax.axis("off")
        elif "raiFilteredReason" in item:
            rai_reason = item["raiFilteredReason"]
            current_ax.text(
                0.5,
                0.5,
                rai_reason,
                horizontalalignment="center",
                verticalalignment="center",
                transform=current_ax.transAxes,
                fontsize=12,
                wrap=True,
            )
            current_ax.set_xlim(0, 1)
            current_ax.set_ylim(0, 1)
            current_ax.axis("off")

    plt.tight_layout()
    plt.show()


# Download image bytes from a GCS URI.
def download_gcs_image_bytes(uri: str) -> bytes:
    matched = re.match(r"gs://(.*?)/(.*)", uri)

    if matched:
        bucket_name = matched.group(1)
        object_name = matched.group(2)
    else:
        raise ValueError(f"Invalid GCS URI format: {uri}")

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(object_name)
    return blob.download_as_bytes()


# Constructs a Vertex AI PredictRequest and uses it to call Virtual Try-On.


def call_virtual_try_on(
    person_image_bytes=None,
    product_image_bytes=None,
    person_image_uri=None,
    product_image_uri=None,
    sample_count: int = 1,
    base_steps=None,
    safety_setting=None,
    person_generation=None,
) -> PredictResponse:
    instances = []

    if person_image_uri and product_image_uri:
        instance = {
            "personImage": {"image": {"gcsUri": person_image_uri}},
            "productImages": [{"image": {"gcsUri": product_image_uri}}],
        }
        instances.append(instance)
    elif person_image_bytes and product_image_bytes:
        instance = {
            "personImage": {"image": {"bytesBase64Encoded": person_image_bytes}},
            "productImages": [{"image": {"bytesBase64Encoded": product_image_bytes}}],
        }
        instances.append(instance)
    else:
        raise ValueError(
            "Both person_image_bytes and product_image_bytes or both person_image_uri and product_image_uri must be set."
        )

    parameters = {"sampleCount": sample_count}

    if base_steps:
        parameters["baseSteps"] = base_steps

    if safety_setting:
        parameters["safetySetting"] = safety_setting

    if person_generation:
        parameters["personGeneration"] = person_generation

    start = timeit.default_timer()

    response = client.predict(
        endpoint=model_endpoint, instances=instances, parameters=parameters
    )
    end = timeit.default_timer()
    print(f"Virtual Try-On took {end - start:.2f}s.")

    return response

## Call the Virtual Try-On API

You can call the API by forming requests with input images either as Cloud Storage URIs or uploaded as base64 bytes.

### Request with image bytes

In [ ]:
# @title Set a person image
# @markdown Run this cell to enable and select the Choose files button. You can then select an image file from your local device to upload. Large images are resized to a maximum dimension of 1024 pixels for faster processing.


images = files.upload()
RAW_PERSON_IMAGE_BYTES = list(images.values())[0]
ENCODED_PERSON_IMAGE_BYTES = base64.b64encode(RAW_PERSON_IMAGE_BYTES).decode("utf-8")
PERSON_IMAGE_PIL = Image.open(io.BytesIO(RAW_PERSON_IMAGE_BYTES)).convert("RGB")
PERSON_IMAGE_PIL.thumbnail((1024, 1024))
plt.axis("off")
plt.imshow(PERSON_IMAGE_PIL)

print(
    f"image size(with x height): {PERSON_IMAGE_PIL.size[0]} x {PERSON_IMAGE_PIL.size[1]}"
)

In [ ]:
# @title Set a product image
# @markdown Run this cell to enable and select the Choose files button. You can then select an image file from your local device to upload. Large images are resized to a maximum dimension of 1024 pixels for faster processing.
images = files.upload()
RAW_PRODUCT_IMAGE_BYTES = list(images.values())[0]
ENCODED_PRODUCT_IMAGE_BYTES = base64.b64encode(RAW_PRODUCT_IMAGE_BYTES).decode("utf-8")
PRODUCT_IMAGE_PIL = Image.open(io.BytesIO(RAW_PRODUCT_IMAGE_BYTES)).convert("RGB")
PRODUCT_IMAGE_PIL.thumbnail((1024, 1024))
plt.axis("off")
plt.imshow(PRODUCT_IMAGE_PIL)

print(
    f"image size(with x height): {PRODUCT_IMAGE_PIL.size[0]} x {PRODUCT_IMAGE_PIL.size[1]}"
)

#### Send request

In [ ]:
# Parameters
sample_count = 1  # 1-4
base_steps = None
safety_setting = "block_low_and_above"  # ["block_low_and_above", "block_medium_and_above", "block_only_high", "block_none"]
person_generation = "allow_adult"  # ["dont_allow", "allow_adult", "allow_all"]

response = call_virtual_try_on(
    person_image_bytes=ENCODED_PERSON_IMAGE_BYTES,
    product_image_bytes=ENCODED_PRODUCT_IMAGE_BYTES,
    sample_count=sample_count,
    base_steps=base_steps,
    safety_setting=safety_setting,
    person_generation=person_generation,
)

display_row(list(response.predictions))

### Request with Cloud Storage URIs

In [ ]:
# @title Set input image URIs
# @markdown Enter `gs://...` paths to image files in Cloud Storage and then run \
# @markdown this cell. Make sure your selected project has access to these URIs.

PERSON_IMAGE_URI = ""  # @param {'type': 'string'}
PRODUCT_IMAGE_URI = ""  # @param {'type': 'string'}
pil_images = []
for uri in [PERSON_IMAGE_URI, PRODUCT_IMAGE_URI]:
    product_image_bytes = download_gcs_image_bytes(uri)
    product_image = Image.open(io.BytesIO(product_image_bytes)).convert("RGB")
    pil_images.append(product_image)

print("Previewing images from GCS: ")
display_row(pil_images)

#### Send request

In [ ]:
sample_count = 1  # 1-4
base_steps = None
safety_setting = "block_low_and_above"  # ["block_low_and_above", "block_medium_and_above", "block_only_high", "block_none"]
person_generation = "allow_adult"  # ["dont_allow", "allow_adult", "allow_all"]

response = call_virtual_try_on(
    person_image_uri=PERSON_IMAGE_URI,
    product_image_uri=PRODUCT_IMAGE_URI,
    sample_count=sample_count,
    base_steps=base_steps,
    safety_setting=safety_setting,
    person_generation=person_generation,
)

display_row(list(response.predictions))